# 🎶 Objectif 5 : Impact des BPM et de la Liveness sur le Succès des Hits Musicaux des dernières années 

#### Rappel

Cet objectif se concentre sur la relation entre les fréquences BPM (battements par minute) et les pourcentages de "liveness" (présence en concert ou en direct) dans la musique populaire des dernières années. En combinant des outils visuels comme une jauge BPM pour réguler les fréquences et une grille filtrable 10x10 pour sélectionner des plages de pourcentages de liveness, ce graphique met en lumière 2 caractéristiques clés qui influencent le succès des morceaux.

**Techniques utilisées :**  
- **Jauge BPM interactive** : Permet de réguler la plage de fréquences BPM à afficher.  
- **Grille filtrable 10x10** : Permet de sélectionner une plage de pourcentage de liveness pour affiner l'analyse.  
- **Visualisation de la tendance des streams** : Diagramme en ligne avec possibilité de zoomer sur des périodes spécifiques (focus + context). 
- **But** : Montrer l'importance des BPM et de la liveness dans la composition des hits musicaux


## Import et configuration

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go
import colorsys

In [4]:


def create_color_gradient(value, max_value=100):
    percentage = value / max_value
    h, s = 210, 100  # Constant hue (blue) and saturation
    l = 90 - (percentage * 60)  # Light to dark gradient
    rgb = [int(x * 255) for x in colorsys.hls_to_rgb(h/360, l/100, s/100)]
    return f'rgb({rgb[0]},{rgb[1]},{rgb[2]})'

# Load and prepare data
df = pd.read_csv('dataset/dataset_filtered.csv')
for col in ['liveness', 'bpm', 'streams']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df['date'] = pd.to_datetime(dict(
    year=df['releasedyear'],
    month=df['releasedmonth'],
    day=df['releasedday']
))

# Filter for popular tracks (>10M streams)
popular_tracks = df.groupby('track')['streams'].sum() >= 10_000_000
df_filtered = df[df['track'].isin(popular_tracks[popular_tracks].index)].copy()

# App layout components
grid_style = {
    'display': 'grid',
    'gridTemplateColumns': 'repeat(10, 1fr)',
    'gridTemplateRows': 'repeat(10, 1fr)',
    'gap': '3px',
    'width': '100%',
    'height': '300px',
    'margin': 'auto',
    'backgroundColor': '#f0f0f0',
    'padding': '10px',
    'borderRadius': '10px',
    'boxSizing': 'border-box'
}

grid_cell_style = {
    'width': '100%',
    'height': '100%',
    'position': 'relative',
    'border': '1px solid white'
}

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("BPM et Liveness : Clés de la Popularité Musicale", 
            style={'textAlign': 'center', 'color': '#2C3E50', 'marginBottom': '30px'}),
    
    html.Div([
        # BPM Section
        html.Div([
            html.H3("Filtre BPM (min-max)"),
            dcc.RangeSlider(
                id='bpm-slider',
                min=0, max=210, step=1, value=[0, 210],
                marks={i: str(i) for i in [0, 70, 140, 210]}
            ),
            dcc.Graph(id='bpm-gauge', style={'height': '300px'}),
            html.Div(id='bpm-text', style={'textAlign': 'center', 'fontSize': '1.2em', 'marginTop': '10px'})
        ], style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
        
        # Liveness Section
        html.Div([
            html.H3("Filtre Liveness (min-max)"),
            dcc.RangeSlider(
                id='liveness-slider',
                min=0, max=100, step=1, value=[0, 100],
                marks={i: f'{i}%' for i in range(0, 101, 25)}
            ),
            html.Div(id='liveness-grid', style=grid_style),
            html.Div(id='liveness-text', style={'marginTop': '20px', 'fontSize': '1.2em'})
        ], style={'width': '50%', 'display': 'inline-block', 'padding': '10px'})
    ], style={'display': 'flex', 'alignItems': 'center'}),
    
    dcc.Graph(id='streams-chart'),
    html.Div(id='filter-info', style={
        'marginTop': '20px',
        'padding': '10px',
        'backgroundColor': '#f8f9fa',
        'borderRadius': '5px',
        'textAlign': 'center'
    })
])

@app.callback(
    [Output('streams-chart', 'figure'),
     Output('bpm-gauge', 'figure'),
     Output('liveness-grid', 'children'),
     Output('bpm-text', 'children'),
     Output('liveness-text', 'children'),
     Output('filter-info', 'children')],
    [Input('bpm-slider', 'value'),
     Input('liveness-slider', 'value')]
)
def update_visuals(bpm_range, liveness_range):
    # Filter data
    current_filter = df_filtered[
        (df_filtered['bpm'].between(bpm_range[0], bpm_range[1])) &
        (df_filtered['liveness'].between(liveness_range[0], liveness_range[1]))
    ]
    
    monthly_streams = current_filter.groupby(
        ['date', 'track', 'artistname', 'bpm', 'liveness']
    )['streams'].sum().reset_index()

    # Create streams chart
    fig_streams = go.Figure()
    if not monthly_streams.empty:
        fig_streams.add_trace(go.Scatter(
            x=monthly_streams['date'],
            y=monthly_streams['streams'],
            mode='markers',
            marker=dict(color='#2E86C1', size=8, opacity=0.7),
            hovertemplate=(
                "Date: %{x|%d/%m/%Y}<br>"
                "Streams: %{y:,.0f}<br>"
                "Titre: %{customdata[0]}<br>"
                "Artiste: %{customdata[1]}<extra></extra>"
            ),
            customdata=monthly_streams[['track', 'artistname']].values
        ))
    
    fig_streams.update_layout(
        title="Dynamique des streams (filtrés)",
        xaxis_title="Date",
        yaxis_title="Nombre de streams (échelle logarithmique)",
        yaxis_type='log',
        xaxis_range=['2020-01-01', monthly_streams['date'].max().strftime('%Y-%m-%d')] 
            if not monthly_streams.empty else None,
        height=400,
        template='plotly_white'
    )

    # Create BPM gauge
    bpm_fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=sum(bpm_range) / 2,
        domain={'x': [0, 1], 'y': [0, 1]},
        gauge={
            'axis': {'range': [0, 210]},
            'bar': {'color': "#FFFFFF"},
            'bgcolor': "white",
            'borderwidth': 2,
            'bordercolor': "gray",
            'steps': [
                {'range': [0, 70], 'color': '#87ceeb'},
                {'range': [70, 140], 'color': '#2E86C1'},
                {'range': [140, 210], 'color': '#1B4F72'}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': bpm_range[1]
            }
        }
    ))
    bpm_fig.update_layout(font={'size': 16}, margin=dict(l=20, r=20, t=50, b=20), height=300)

    # Create liveness grid
    grid_squares = [
        html.Div(style={
            **grid_cell_style,
            'backgroundColor': create_color_gradient(i) if liveness_range[0] <= i <= liveness_range[1] else '#E0E0E0'
        }) for i in range(100)
    ]

    return (
        fig_streams,
        bpm_fig,
        grid_squares,
        f"BPM Range: {bpm_range[0]} - {bpm_range[1]}",
        f"Liveness Range: {liveness_range[0]}% - {liveness_range[1]}%",
        f"Affichage de {len(current_filter['track'].unique())} morceaux correspondant aux critères sélectionnés"
    )

if __name__ == '__main__':
    app.run_server(debug=True, port=8057)